<a href="https://colab.research.google.com/github/prithwis/Centaur/blob/main/Vikram_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://raw.githubusercontent.com/prithwis/Centaur/refs/heads/main/images/CentaurBanner2.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Vikram
A basic LLM based adjudicator for a Matrix (Geopolitical) Strategy Game

#Install & Test Setup

Install + load key<br>
OPENAI_API_KEY needs to defined as a Colab "secret" for the Google ID used to run this notebook

In [1]:
!pip install --quiet openai

In [2]:
import os
import requests
from google.colab import userdata

try:
    # Load key from Colab Secrets into environment
    api_key = userdata.get("OPENAI_API_KEY")
    if not api_key:
        raise ValueError("OPENAI_API_KEY not found in Colab Secrets")

    os.environ["OPENAI_API_KEY"] = api_key
    print("API key loaded ✔")

    # Call OpenAI identity endpoint
    headers = {
        "Authorization": f"Bearer {api_key}"
    }

    resp = requests.get("https://api.openai.com/v1/me", headers=headers)
    resp.raise_for_status()   # catches 401 / 403 / etc.

    me = resp.json()
    name = me.get("name", "N/A")
    email = me.get("email", "N/A")

    print(f"Logged in as {name} {email}")

except Exception as e:
    print("❌ OpenAI credential check failed")
    print("Reason:", str(e))


API key loaded ✔
Logged in as Calcutta prithwis@yantrajaal.com


#Core OpenAI LLM API Call

In [3]:
#| Model            | Best For                  | Notes                                       |
#| ---------------- | ------------------------- | ------------------------------------------- |
#| **GPT-4.1**      | Highest-quality reasoning | Ideal judge for complex scenario evaluation |
#| **GPT-4.1-mini** | Balanced reasoning & cost | Strong choice for adjudication logic        |
#| **GPT-4.1-nano** | High volume, low cost     | Good for simple reasoning tasks             |
#| **gpt-4o-mini**  | Prototyping & cheap       | Great starter, but upgrade recommended      |



from openai import OpenAI

OpenAI_client = OpenAI()

def OpenAI_llm_call(system_prompt, user_prompt, model="gpt-4o-mini"):          #Great starter, but upgrade recommended
    response = OpenAI_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.2,
    )
    #return response.choices[0].message.content.strip()

    return {
    "header": f"OpenAI {model}",
    "content": response.choices[0].message.content.strip(),
    "usage": response.usage
    }


In [4]:
def parse_usage(usage):

    completion_tokens = usage.completion_tokens
    prompt_tokens = usage.prompt_tokens
    total_tokens = usage.total_tokens

    #print(completion_tokens, prompt_tokens, total_tokens)
    return(f" | token usage {completion_tokens} + {prompt_tokens} = {total_tokens}")

In [5]:
import textwrap

def reveal(result: dict, width: int = 80):
    import textwrap

    #token_usage = parse_usage(result.get("usage"))
    print("Response from ", result.get("header"), parse_usage(result.get("usage")),"\n")
    print("-" * width)

    for paragraph in result.get("content", "").strip().split("\n\n"):
        print(textwrap.fill(paragraph.strip(), width=width))
        print()


#Define Decision Maker
Specifies Role

Named after [Raja Vikramaditya of Ujjain](https://en.wikipedia.org/wiki/Vikramaditya)

In [6]:
def VikramDecision(
    background_info: str,
    trigger_event: str,
    response: str
):

    ROLE = """
    You are an adjudicator in a Matrix Strategy Game.

    Definition:
    - A Matrix Strategy Game evaluates player actions against a structured state space
    including political, military, economic, and environmental dimensions.
    - Player actions are proposals, not guaranteed outcomes.
    - The adjudicator does not simulate optimally; it evaluates plausibility and impact.

    Adjudication Principles:
    - Do not invent rules or hidden mechanics.
    - Do not assume success of any action.
    - Assess second-order effects conservatively.
    - Consider proportionality between action and outcome.
    - If information is insufficient, state uncertainty explicitly.

    Output Rules:
    - Provide a clear outcome assessment.
    - Provide concise reasoning.
    - Do not introduce new facts not present in the context.

    """

    context = f"""
BACKGROND :
{background_info}

TRIGGER EVENT :
{trigger_event}

PROPOSD RESPONSE :
{response}

Adjudicate the proposed response strictly under the stated principles.

Respond with:
1. Outcome
2. Reasoning
"""

    llm_response = OpenAI_llm_call(ROLE, context)
    llm_response["header"] = "Vikram w/ "+llm_response["header"]

    return(llm_response)


#Geopolitical Background / Business Case
Background + Trigger Event

In [7]:
Scenario = """
Rare Earth Elements (REEs) are critical inputs for advanced manufacturing,
including electric motors, precision guidance systems, renewable energy
infrastructure, and consumer electronics.

While REEs are geologically widespread, economically viable extraction,
separation, and refining are highly concentrated. China dominates global
REE processing and refining capacity, particularly for neodymium and
praseodymium used in high-performance permanent magnets.

India possesses limited REE reserves and minimal downstream refining
capability, relying heavily on imports for both raw materials and
finished magnet components. Efforts to diversify supply have been slow
due to environmental concerns, capital intensity, and technological barriers.

Global REE supply chains are politically sensitive, opaque, and prone to
state intervention.
"""

Crisis = """
The Chinese government announces an immediate suspension of neodymium
exports to India, citing regulatory reviews and domestic supply
prioritization. No explicit timeline for resumption is provided.

The suspension applies to both refined neodymium oxide and downstream
magnet-grade materials. Other export destinations are not formally
restricted, though delays are reported.

Indian manufacturers report inventory coverage of approximately
8–12 weeks, with defense and electric vehicle supply chains most exposed.
"""


#Responses
Obtained by "calling" Adjudicator - currently VikramDecision()

##Option 1

In [8]:
Response = """
Explore indirect procurement through international trading houses and
non-transparent spot markets to bridge short-term shortages, while avoiding
formal policy announcements. Accept higher costs and limited volumes as a
temporary measure.

Simultaneously signal willingness to negotiate technical and regulatory
concerns raised by China, without making public concessions.
"""



reveal(VikramDecision(Scenario, Crisis, Response))


Response from  Vikram w/ OpenAI gpt-4o-mini  | token usage 315 + 508 = 823 

--------------------------------------------------------------------------------
1. **Outcome**: The proposed response may provide a temporary solution to short-
term shortages of neodymium for Indian manufacturers, but it carries significant
risks and uncertainties.

2. **Reasoning**:     - **Indirect Procurement**: Exploring indirect procurement
through international trading houses and non-transparent spot markets could
potentially alleviate immediate supply shortages. However, this approach is
likely to result in higher costs and limited volumes, as indicated in the
proposal. The effectiveness of this strategy is uncertain due to the opaque
nature of these markets and potential legal or regulatory repercussions.
- **Negotiation Signals**: Signaling a willingness to negotiate with China may
help in creating a more favorable environment for future discussions. However,
without public concessions, the impact o

##Option 2

In [9]:
Response = """
Treat the suspension as a strategic inflection point rather than a short-term
supply shock. Publicly acknowledge dependence, absorb near-term production
losses, and initiate an accelerated national program to build domestic REE
processing and magnet manufacturing capacity.

This includes emergency environmental clearances, state-backed capital
deployment, and strategic partnerships with non-Chinese producers, even at
high initial inefficiency. File a formal trade dispute to internationalize
the issue, accepting retaliation risk.

The objective is not immediate supply restoration but irreversible reduction
of structural vulnerability over a multi-year horizon.
"""

reveal(VikramDecision(Scenario, Crisis, Response))

Response from  Vikram w/ OpenAI gpt-4o-mini  | token usage 375 + 564 = 939 

--------------------------------------------------------------------------------
1. **Outcome**: The proposed response is plausible but carries significant risks
and uncertainties.

2. **Reasoning**:     - **Strategic Inflection Point**: Treating the suspension
as a long-term issue is reasonable given India's heavy reliance on Chinese REEs.
However, the effectiveness of this perspective depends on the ability to
mobilize resources and implement changes swiftly, which is uncertain.    -
**Public Acknowledgment and Production Losses**: Acknowledging dependence may
foster transparency but could also impact market confidence and investor
sentiment negatively. Absorbing production losses may be necessary, but the
extent of these losses and their impact on key sectors (defense and electric
vehicles) is not quantified, leading to uncertainty about the short-term
economic consequences.    - **Domestic Capacity Buildin

In [10]:
from datetime import datetime
import pytz
print('Tested on  ',datetime.now(pytz.timezone('Asia/Kolkata')))

Tested on   2026-02-08 18:26:47.476184+05:30


#Chronobooks <br>
Three science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chrono3) <br>
![alt text](https://blogger.googleusercontent.com/img/a/AVvXsEjsZufX_KYaLwAnJP6bUxvDg5RSPn6r8HIZe749nLWX3RuwyshrYEAUpdw03a9WIWRdnzA9epwJOE05eDJ0Ad7kGyfWiUrC2vNuOskb2jA-e8aOZSx8YqzT8mfZi3E4X1Rz3qlEAiv-aTxlCM976BEeTjx4J64ctY3C_FoV4v9aY_U23F8xRqI5Eg=s1600)